# Joint inversion of source location and subsurface structure from body waves (P,S) and Rayleigh Waves (LR) arrival times
Froment, M., Brissaud, Q., Näsholm, S. P. and Schweitzer, J. (2025) _Joint source and subsurface inversion using earthquake-generated infrasound recorded by balloons_

This notebook presents the different steps necessary to run the inversion of source and subsurface for the December 14, 2021 $M_w$ 7.3 earthquake in the Flores Sea. The example inversion uses pressure waveforms recorded by four balloons of the Strateole2 campaign flying over Oceania during the event. First, the balloon data is processed to reduced the influence of low-frequency buoyancy oscillations. Then, P, S and LR phases arrival times are extracted manually from the processed pressure waveforms and formatted for the inversion. A short test inversion ($10^3$ steps) is run. The Monte Carlo inversion results are then further processed and relevant figures shown in the article are produced.     

In [10]:
import MCMC_modules as mc
import numpy as np
import sys
import importlib 
# %matplotlib widget
### Necessary to use interactive picking figures 
%matplotlib qt 

# Balloon Data processing
Processing is done through the `prepare_data_flores_Strateole.py` code. The raw balloon data and the processed outputs are saved in `.mseed` format is in the `./Flores_data/` directory.

In [11]:
import prepare_data_flores_Strateole as bp
importlib.reload(bp)
data_source = "./Flores_data/"

skip =True 
if not skip:
    ### STEP 1: INITIALIZE STREAMS 
    bdata = bp.BALLOON_DATA()
    
    ### STEP 2: CORRECT GAPS 
    list_streams = bp.correct_gaps(bdata)
    
    ### STEP 3: UPSAMPLE Z 
    list_streams_aligned = bp.Z_upsample_align(list_streams, do_plot=False)
    
    ### STEP 4: Determine relation between P and Z 
    list_streams_opt = bp.P_Z_relation(list_streams_aligned, do_plot=False)
    
    ### STEP 5: Obtained corrected pressure, plot, save 
    list_streams_corr = bp.P_corrected(list_streams_opt, do_plot=False)
    
    ### STEP 6: Save raw and corrected seeds for inversion 
    bp.save_streams_inversion(data_source, list_streams_corr)

# Phase picking and formatting
In the following step, we will prepare the inversion setup and proceed to pick the arrivals. This step is very important, as it will define the name of our run and where the inputs and outputs of the run will be stored. This step generate a class `DATA`containing the necessary arrays. 

In [12]:
import initialise_inversion_flores_Strateole as data_init
importlib.reload(data_init)
importlib.reload(mc)

data_dir = "./chains_emcee_flores_Strateole/DATA/"
DATA = data_init.generate_data(data_dir, initialize=True)

Building data to invert
Loading recorded data.
Computing FTAN.
Start phase picker... 
Loading recorded data.
Computing FTAN.
Start phase picker... 
Loading recorded data.
Computing FTAN.
Start phase picker... 
Loading recorded data.
Computing FTAN.
Start phase picker... 
Saving MCMC Files


# Preparing the inversion
Now that `DATA` is prepared, we can set up the inversion. This step generate a class `MCMC` that contains functions to initialise, load and run the McMC samplers, and also contains the results and basic functions to post-process them.

### Step 1: Defined the prior
Prior bounds are defined in a separate text file, here named `prior_flores.txt`. Some precautions: 
- The variable names in the first column should not be changed: the first four variables are relative to the source, then the subsurface velocity parameters follow.
- This files defines the number of layers that will be inverted. There should always be one more line for `vs` and `poisson` than for `h_layer`. If there are 6 lines for `h_layer`, then it means a model with 6 layers and 1 halfspace will be inverted.
- The two columns `prior_min` and `prior_max` define uniform prior bounds for each variable. The two columns `start_min` and `start_max` define restricted bounds, which help initialise the Monte Carlo inversion in a parameter region that is not too pathological. The starting bounds must always be comprised withing the prior bounds, or an error will be raised.

### Step 2: Define variables for inversion

In [13]:
### Name of the run: must be the same as in "data_dir"
name_run      = "flores_Strateole"

### Directory to save results. Must be the same as in "data_dir"
save_dir      = data_dir.split("DATA/")[0]
data_dir      = data_dir

### File containing the priors
param_file    = "prior_flores.txt"

### McMC sampler. emcee is the most optimized option here.
method        = "emcee"

### Number of desired iterations
n_iter        = int(1e2)

### Number of CPUs used. 1 preferred for debugging. 
n_cpus        = int(10) 

### To display progress bar (must be set to false if running through a cluster) 
progress      = True

### If set to True, generate a brand new inversion. 
### If set to False, restarts an inversion from the last steps stored in "save_dir"
reset_backend = True

importlib.reload(mc)
MCMC = mc.MCMC_Model(DATA, name_run, save_dir, data_dir, param_file, 
                    method=method, n_iter=n_iter, n_cpus=n_cpus, 
                    progress=progress, reset_backend=reset_backend)

24 CPUs available
10 CPUs running
doing emcee...
Deleting previous backend...


# Run the McMC sampler

Diagnostic information about run (timing, CPUs, crash, reason for crash) are stored in the file `timing_[method].txt`. 

In [14]:
### Will do a warm start if reset_backend is False. 
MCMC.run()

Reset start
  2%|▊                                          | 2/100 [00:00<00:10,  9.04it/s]

/nobackup/marouchka/miniconda/envs/env_mcmc/lib/python3.13/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in scalar subtract
  lnpdiff = f + nlp - state.log_prob[j]


100%|█████████████████████████████████████████| 100/100 [00:12<00:00,  8.12it/s]
- SUCCESS -


# Visualize results

The `MCMC` class possesses simple plotting functions to analyse inversion results. 
### General inspection
The first function is called `inspect_chain()` and allows to inspect the behavior of the chain during the inversion. It will display: 
- The evolution of the log_likelihood during the inversion. This plot can be used to identify "stuck" chains and the `crit` parameter (see below)
- The evolution of the acceptance rate
- The evolution of each inverted parameter (if using `emcee`, there are multiple curves for each walker and the least-misfit walker is plotted in red), the histogram of the first parameter and an estimate of the autocorrelation time of the chain for this parameter.

The `thin` parameter controls the amount of chain points that should be discarded. It is here set to display only 1000 chain iterations.

In [15]:
importlib.reload(mc)
MCMC._load_results()
# MCMC.inspect_chain(param = 0, direction = "horizontal", thin=max(1,int(n_iter//1000)), do_save=False)

### Autocorrelation and convergence

This function calculates the autocorrelation time $\tau$ of chains to evaluate their convergence, following https://emcee.readthedocs.io/en/stable/tutorials/autocorr/. A chain is considered converged when its autocorrelation time becomes stable. `param` is the parameter index from whose chain the autocorrelation will be estimated. `discard` allows to throw away the burn-in iterations before calculating the autocorrelation time.

In [16]:
MCMC.visualize_convergence(discard=1, N_point_autocorr = 50, param = 0, do_save=True)

Loading data...
Calculating autocorrelation times...
100%|██████████████████████████████████████████| 49/49 [00:00<00:00, 147.67it/s]


### Source and subsurface solutions 

The following function plot some diagnostic information on the inverted source location and time, and the posterior subsurface models. Before being run, they require the solutions to be processed: some _burn-in_ iterations will be discarded (option `discard`) and only a selection of iterations will be plotted (option `thin`).

Chains that could be "stuck" in a region of very low likelihood can be discarded as well using a threshold on the log-likelihood with the option `crit`. 

In [17]:
crit = -9999
importlib.reload(mc)
MCMC.process_solutions(discard=0, thin=1, crit=crit)
### For a very large number of iterations:
#MCMC.process_solutions(discard=1000, thin=2000, crit=crit)

### SHALLOW Subsurface
# MCMC.visualize_solution(figsize=(10,5), Nmod=200, bins_hist=20, bins_wave=50, hspace=0.1, wspace=0.15,  
#                             depth_interpolation=np.linspace(0., 100., 201), preserve_aspect=True, do_save=True)
### DEEPER subsurface
# MCMC.visualize_solution(figsize=(10,5), Nmod=200, bins_hist=20, bins_wave=50, hspace=0.1, wspace=0.15,  
#                             depth_interpolation=np.linspace(0., 1000., 201), preserve_aspect=True, do_save=True)

### Source location 
MCMC.source_location(figsize=(4,4), do_save=True, geography = True, zoom=True)

Loading samples...
Extracted 2178 subsurface models


### Marginal distributions of parameters
The following two function make corner plots of all or a selection of couples of parameters. This allows to visualise trade-off between two dimensions. 

In [21]:
importlib.reload(mc)

### VERY BIG CORNER PLOT 
# MCMC.corner_plot(do_save=True, do_MAP = True)

### Marginals for a selection of parameters 
#MCMC.parameter_marginals(pars=[[0,1],[0,2],[1,8],[2,8],[5,8],[4,14],[0,8]], find_limits=True, 
#                        do_save=True, do_MAP=True)

<module 'MCMC_modules' from '/staff/marouchka/Documents/SOFTWARE/McMC/QB/public_inversion/MCMC_modules.py'>

### Additional information: the Priors
The prior distribution of subsurface and source parameters can be plotted with the following function.

In [22]:
importlib.reload(mc)

# MCMC.visualize_priors(figsize=(10,5), Nmod=2000, bins_hist=20, bins_wave=50, hspace=0.1, wspace=0.15, 
#                            depth_interpolation=np.linspace(0., 150., 201), do_save=False, preserve_aspect=True)

<module 'MCMC_modules' from '/staff/marouchka/Documents/SOFTWARE/McMC/QB/public_inversion/MCMC_modules.py'>